In [110]:
import clip
import torch
import torch.nn.functional as F
from torchvision import transforms
from torchvision.utils import save_image
from PIL import Image
import matplotlib.pyplot as plt
from forgery import *
from torchvision.utils import save_image
import torch.nn as nn
import numpy as np
from types import SimpleNamespace
from stable_signature_util import *

In [111]:
msg_extractor = torch.jit.load("models/dec_48b_whit.torchscript.pt").to("cuda")
key = '111010110101000001010111010011010100010000100111' 

orig_path = "/work/forgery/Data/StableSignature2/watermarked/img_5976.png"
to_tensor = transforms.Compose([
    transforms.ToTensor(),  
])

img = to_tensor(Image.open(orig_path)).unsqueeze(0).float()
mes = watermark_decoder(msg_extractor,img)
computer_acc(mes,key)

1.0

In [112]:
from hidden.models import HiddenDecoder
# print(msg_extractor)
decoder = HiddenDecoder(num_blocks=8, num_bits=48, channels=64).cuda()

# 提取第一个子模块（HiddenDecoder）
hidden_decoder_jit = list(msg_extractor.children())[0]

# 提取其权重
state_dict = hidden_decoder_jit.state_dict()

# 加载进你定义的 decoder
decoder.load_state_dict(state_dict, strict=True)


linear_head_jit = list(msg_extractor.children())[1]  # 是一个 Linear(48, 48)

# 构建对应结构
linear_head = nn.Linear(48, 48).cuda()

# 加载权重
linear_head.load_state_dict(linear_head_jit.state_dict(), strict=True)

<All keys matched successfully>

In [113]:
msg_extractor.eval()
full_model.eval()
with torch.no_grad():
    mes = watermark_decoder(msg_extractor,img)
    print(computer_acc(mes,key))
with torch.no_grad():
    full_model = nn.Sequential(decoder, linear_head).cuda()
    output = watermark_decoder(full_model,img)
    acc = computer_acc(output,key)
    print(acc)

1.0
0.9791666666666666
